# Récupération des données historiques de l'indice VIX et de l'indice de performance du SP500 

**Projet Python - 2A ENSAE** . 

Elena Loumagne / Jérémie Darracq 



## Introduction 
Ce notebook a été crée pour webscrapper les données de l'indice VIX et de l'indice de performance du SP500. Pour obtenir le jeu de données entier, nous allons utiliser la librairie **selenium** qui nous permettra de naviguer sur page HTML. 


## Packages utilisés 

In [4]:
#%pip install selenium 

In [1]:
import time 
import bs4
import pandas
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime


## WEB SCRAPPING 

On crée une fonction de webscrapping utilisable sur les données du site [Investing.com](https://fr.investing.com/indices/) : 

- Étape 1 : on accède au site web en acceptant les cookies   
- Étape 2 : on choisit la période sur laquelle on veut afficher les valeurs de l'indice du VIX     
- Étape 3 : on stocke la page Html pour pouvoir ensuite extraire les données 

In [2]:
def web_scrapper(path,Xpath):
    """
    Paramètres :
    -------------

    'path : str
    Lien url de la page web 
    
    'Xpath : str
    Lien Xpath de l'objet date à modifier 

    Return :
    ---------

    'webpage : code HTML de la page webscrapée 
    """

    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(path)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))).click()

    dropdown =driver.find_element("id",'history-timeframe-selector')
    dropdown.click() 

    # on cherche les différentes options présentes dans le dropdown et on clique sur celle qu'on veut
    driver.find_element("id","react-select-2-option-0").click() # 0 -> journalier, 1 -> Hebdomadaire, 2 -> pour Mensuel 

    calendar=driver.find_element(By.CLASS_NAME,"historical-data_history-date-picker-wrapper__dDOuq")
    calendar.click()

    date_input = driver.find_element(By.XPATH,Xpath)
    date_input.clear()
    date_input.send_keys("3004-08-2005") #date à laquelle on souhaite commencer (marche pas mais permet de donner tout l'historique pour l'instant)


    validate=driver.find_element(By.CLASS_NAME,"HistoryDatePicker_arrow-icon__NwxN4")
    validate.click()
    time.sleep(6)

    time.sleep(6)

    webpage=driver.page_source # on stock le code html de la page où toutes le données sont chargées
    driver.quit()  
    print("Page Webscrappée")

    return webpage

### Webscrapping de l'indice VIX 

In [38]:
path_VIX= 'https://fr.investing.com/indices/volatility-s-p-500-historical-data'
Xpath_VIX = '//*[@id="__next"]/div/div/div/div[2]/main/div/div[5]/div/div/div[2]/div[2]/div[2]/div[2]/div[1]/div/div[1]/input'

web_page_VIX = web_scrapper(path_VIX,Xpath_VIX)


<ipython-input-35-61410f1f3154>:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Page Webscrappée


### Webscrapping de l'indice de performance du SP500

In [3]:
path_SP500 = 'https://fr.investing.com/indices/us-spx-500-historical-data'
Xpath_SP500 = '//*[@id="__next"]/div/div/div/div[2]/main/div/div[5]/div/div/div[2]/div[2]/div[2]/div[2]/div[1]/div/div[1]/input'

web_page_SP500 = web_scrapper(path_SP500,Xpath_SP500)


[WDM] - Downloading: 100%|██████████| 6.58M/6.58M [00:40<00:00, 172kB/s] 
C:\Users\elelo\AppData\Local\Temp/ipykernel_8280/3617288181.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Page Webscrappée


## Passage de l'HTML au Dataframe 


Maintenant que nous avons le code HTML de toutes les données, nous allons décortiquer la page grâce à la librairie BeautifulSoup.

On crée une fonction pour passer du code source Html à un Dataframe. Cette fonction récupère les valeurs de l'indice affichées dans le tableau des données historiques.

In [4]:
def Html_to_df(web_page):
    """
    'web_page : html code 
     code html de la page webscrappée
    """
    # utiliser le package BeautifulSoup qui "comprend" les balises 
    page = bs4.BeautifulSoup(web_page, "lxml")

    # on identifie le tableau des indices vix et SP500 : c'est le premier qui a cette classe "datatable_table__D_jso datatable_table--border__B_zW0 datatable_table--mobile-basic__W2ilt datatable_table--freeze-column__7YoIE"
    tableau_indice = page.find('table', {'class' : 'datatable_table__D_jso datatable_table--border__B_zW0 datatable_table--mobile-basic__W2ilt datatable_table--freeze-column__7YoIE'})
    table_body = tableau_indice.find('tbody')

    # on recherche toutes les lignes du tableau avec la balise "tr"
    rows = table_body.find_all('tr')

    # on obtient une liste où chaque élément est une des lignes du tableau
    cols = rows[0].find_all('td')
    dico_indice = dict()
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        if len(cols) > 0 : 
            dico_indice[cols[0]] = cols[1:]
    
    # On transforme le dictionnaire en DataFrame  
    data_indice = pandas.DataFrame.from_dict(dico_indice,orient='index')        
    return data_indice 


    

### DataFrame de l'indice du VIX

In [58]:
data_vix = Html_to_df(web_page_VIX)

### DataFrame de l'indice de performance du SP500

In [5]:
data_SP500 = Html_to_df(web_page_SP500)

## Modification des DataFrames

In [6]:
def replace_comma_to_float(text):
    return float(text.replace(',','.'))
    
def replace_point(text):
    return text.replace('.','')


# Fonction principale qui nettoie le dataframe 

def clear_df(data,indice):
    ## On renomme les colonnes du dataframe
    data = data.rename(columns={0:"dernier_"+indice,1:'ouverture',2:'higher',3:'lower',4:'volatilité',5:'variation'})
    data = data.drop(columns=['volatilité'])
    
    ## Pour pouvoir utiliser nos données, il faut transformer les indices en *float*
    if indice=='SP500':
        data["dernier_"+indice]=data["dernier_"+indice].apply(replace_point)
        data["ouverture"]=data["ouverture"].apply(replace_point)
        data["higher"]=data["higher"].apply(replace_point)
        data["lower"]=data["lower"].apply(replace_point)

    data["dernier_"+indice]=data["dernier_"+indice].apply(replace_comma_to_float)
    data["ouverture"]=data["ouverture"].apply(replace_comma_to_float)
    data["higher"]=data["higher"].apply(replace_comma_to_float)
    data["lower"]=data["lower"].apply(replace_comma_to_float)
    
    ## On modifie le format de la date 
    data["date"]=data.index
    data.reset_index(drop=True, inplace=True)
    data["date"]=data['date'].apply(lambda x : datetime.strptime(x, '%d/%m/%Y'))
    return data



### CSV final indice du VIX 

In [61]:
data_vix = clear_df(data_vix,'VIX')
data_vix.to_csv("Data/data_vix.csv",index=False)
data_vix.head()

,dernier_VIX,ouverture,higher,lower,variation,date
0,22.83,22.68,22.89,22.68,+0.66%,2022-12-08
1,22.68,22.32,23.01,22.18,+2.30%,2022-12-07
2,22.17,20.69,22.60,20.38,+6.84%,2022-12-06
3,20.75,20.30,21.29,19.78,+8.87%,2022-12-05
4,19.06,20.42,20.96,18.95,-3.93%,2022-12-02


### CSV final indice de performance du SP500

In [7]:
data_sp500 = clear_df(data_SP500,'SP500')
data_sp500.to_csv("Data/data_sp500.csv",index=False)
data_sp500.head()

,dernier_SP500,ouverture,higher,lower,variation,date
0,3934.38,4042.00,4051.75,3918.40,-3.37%,2022-12-04
1,4071.70,3994.60,4099.40,3937.50,+1.13%,2022-11-27
2,4026.12,3953.69,4034.00,3932.55,+1.53%,2022-11-20
3,3965.34,3973.63,4029.99,3906.26,-0.69%,2022-11-13
4,3992.93,3786.60,4001.60,3743.85,+5.90%,2022-11-06
